### Connect to workspace

In [1]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /home/demouser/repos/HighPerformanceTensorFlowOnAzure/aml_config/config.json
jkaml
jkaml
eastus2
952a710c-8d9c-40c1-9fec-f752138cc0b3


### Retrieve the model

In [2]:
from azureml.core import Experiment

exp = Experiment(ws, name='aerial-classifier-train')
tags = {"FriendlyName": "FirstRun"}
runs = exp.get_runs(tags=tags)
try:
    run = next(runs)
except:
    print("Could not find the run with {}".format(tags))
    
run.download_file('outputs/aerial_classifier.hd5', 'aerial_classifier.hd5')

### Load the model

In [3]:
import tensorflow.keras as keras
top_model = keras.models.load_model('aerial_classifier.hd5')
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 1,052,166
Trainable params: 1,052,166
Non-trainable params: 0
_________________________________________________________________


### Assemble the model
Stack the *top_model* on top of **ResNet50** "trunk"

In [7]:
from tensorflow.keras.applications import resnet50
from tensorflow.keras import Model

# Create a featurizer
trunk = resnet50.ResNet50(
                weights = 'imagenet', 
                input_shape=(224,224,3), 
                include_top = False,
                pooling = 'avg')

full_model = Model(inputs=trunk.input, outputs=top_model(trunk.output))
full_model.summary()

### Create a dataset

In [39]:
import tensorflow as tf
import numpy as np

# Returns a dataset based on a list of TFRecords files passsed as a parameters. 
def create_dataset(files, batch_size=32, prefetch_buffer_size=1, train=True, buffer_size=10000, num_parallel_calls=4):
    IMAGE_SHAPE = (224, 224, 3,)
    NUM_CLASSES = 6
          
    # Extract image and label from proto Example
    def _parse(example_proto):
        features = {'label': tf.FixedLenFeature((), tf.int64, default_value=0),
                    'image': tf.FixedLenFeature((), tf.string, default_value="")}
        parsed_features = tf.parse_single_example(example_proto, features)
        label = parsed_features['label']
        label = tf.one_hot(label, NUM_CLASSES)
        image = image = tf.decode_raw(parsed_features['image'], tf.uint8)
        image = tf.cast(image, tf.float32)
        image = tf.reshape(image, IMAGE_SHAPE)
                                                                  
        # Pre-process image data for ResNet
        #   RGB -> BGR
        image = image[..., ::-1]
        #   Substract the Imagenet mean for each channel
        imagenet_mean=tf.constant(-np.array([103.939, 116.779, 123.68]), dtype=tf.float32)
        image = tf.nn.bias_add(image, imagenet_mean)
        #image = tf.subtract(image, imagenet_mean)
        #image = resnet50.preprocess_input(image)
        
        return image, label

    dataset = tf.data.TFRecordDataset(files)
    dataset = dataset.map(_parse, num_parallel_calls=num_parallel_calls)
    #dataset = dataset.map(_parse)
    if train:
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(buffer_size=prefetch_buffer_size)
    dataset = dataset.repeat()

    return dataset

In [40]:
def get_num_of_records(files):
    count = 0
    for file in files:
        for record in tf.python_io.tf_record_iterator(file):
            count += 1
    return count

In [41]:
%%sh

ls -l ../datasets/tfrecords/aerialmed/training

total 2598956
-rw-rw-r-- 1 demouser demouser 2661324051 Oct  3 18:50 aerialmed_train.tfrecords


In [42]:
import os

training_dir = '../datasets/tfrecords/aerialmed/training'
validation_dir = '../datasets/tfrecords/aerialmed/validation'

training_files = [os.path.join(training_dir, file) for file in os.listdir(training_dir)]
validation_files = [os.path.join(validation_dir, file) for file in os.listdir(validation_dir)]
                                                                          
print(training_files)
print(validation_files)                                              

['../datasets/tfrecords/aerialmed/training/aerialmed_train.tfrecords']
['../datasets/tfrecords/aerialmed/validation/aerialmed_valid.tfrecords']


In [43]:

from tensorflow.keras.optimizers import SGD

train_dataset = create_dataset(training_files, batch_size = 32, train=True)
valid_dataset = create_dataset(validation_files, batch_size = 32, train=False)

for layer in trunk.layers:
    layer.trainable = False
    

    #optimizer = tf.train.AdadeltaOptimizer()
optimizer = SGD(lr=0.00001, momentum=0.9)
loss = 'categorical_crossentropy'
metrics = ['accuracy']
full_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
full_model.summary()
  



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [46]:
batch_size = 32

steps_per_epoch = get_num_of_records(training_files)//batch_size
validation_steps = get_num_of_records(validation_files)//batch_size

    #print("Starting training:")
    #print("    Epochs: ", FLAGS.epochs)
    #print("    Steps per epoch: ", steps_per_epoch)
    #print("    Validation steps: ", validation_steps)
    #print("    Dropout prob: ", FLAGS.dropout_prob)
    #print("    L1: ", FLAGS.l1)
    #print("    L2: ", FLAGS.l2)

epochs = 10

full_model.fit(train_dataset,
        epochs = epochs,
        steps_per_epoch = steps_per_epoch,
        validation_data = valid_dataset,
        validation_steps = validation_steps)

Epoch 1/10
552/552 [==============================] - 34s 61ms/step - loss: 0.9132 - acc: 0.7139 - val_loss: 0.1834 - val_acc: 0.9596
Epoch 2/10
552/552 [==============================] - 30s 55ms/step - loss: 0.8178 - acc: 0.7515 - val_loss: 0.1904 - val_acc: 0.9583
Epoch 3/10
552/552 [==============================] - 30s 55ms/step - loss: 0.7634 - acc: 0.7737 - val_loss: 0.1965 - val_acc: 0.9577
Epoch 4/10
552/552 [==============================] - 30s 55ms/step - loss: 0.7289 - acc: 0.7829 - val_loss: 0.2021 - val_acc: 0.9567
Epoch 5/10
552/552 [==============================] - 30s 55ms/step - loss: 0.6996 - acc: 0.7929 - val_loss: 0.2087 - val_acc: 0.9564
Epoch 6/10
552/552 [==============================] - 30s 55ms/step - loss: 0.6743 - acc: 0.8016 - val_loss: 0.2123 - val_acc: 0.9564
Epoch 7/10
552/552 [==============================] - 30s 55ms/step - loss: 0.6619 - acc: 0.8074 - val_loss: 0.2168 - val_acc: 0.9561
Epoch 8/10
552/552 [==============================] - 30s 55ms

KeyboardInterrupt: 